In [ ]:
#Compare predictions and true labels

import pandas as pd
import json

#pseudocode:

#read in predicted bounding boxes
pred_df = pd.read_csv('/home/cara/oregon_critters/runs/detect/0_mc/pred_boxes_0mc.csv')

#read in true bounding boxes, keep only columns we need, and remove '.JPG' from image name
true_csv = pd.read_csv('/home/cara/oregon_critters/sampled_ds_300_split.csv')
true_df = true_csv[['IName','CName', 'X','Y','W','H', 'cam_type', 'day_night', 'project', 'group']]
true_df['IName'] = true_df.IName.map(lambda x: x.strip('.JPG'))

#rename columns
true_df.rename(columns={"IName": "image_name","CName": "class_true", "X":"x", "Y":"y", "W":"w", "H":"h"}, inplace=True)


In [ ]:
#Need to add image height and width so I can un-normalize the pred bounding boxes

#load json and turn into dataframe
with open('/home/cara/oregon_critters/dataset_300_split_adjXY.json', 'r') as json_file:
    json_data = json.load(json_file)    

json_df = pd.DataFrame(json_data['images'])

#extract just image name from the filename
json_df['image_name'] = json_df['file_name'].str.extract(r'([^/]+)\.JPG')

#merge with pred_df
pred_df = pd.merge(pred_df, json_df[['image_name', 'width', 'height']], left_on='image_name', right_on='image_name', how='left')

#convert box dimensions from normalized 
pred_df['x_pixel'] = pred_df['x'] * pred_df['width']
pred_df['y_pixel'] = pred_df['y'] * pred_df['height']
pred_df['w_pixel'] = pred_df['w'] * pred_df['width']
pred_df['h_pixel'] = pred_df['h'] * pred_df['height']


In [ ]:
pred_df.head()


In [ ]:
true_df.head()

In [ ]:
# Define function to calculate IOU of two bounding boxes (from Jess):

def calculate_iou(x1, y1, w1, h1, x2, y2, w2, h2):

    # Define bboxes
    box1 = [x1, y1, w1, h1]
    box2 = [x2, y2, w2, h2]

    # Convert bounding box coordinates to the format (x1, y1, x2, y2)
    box1 = [box1[0], box1[1], box1[0] + box1[2], box1[1] + box1[3]]
    box2 = [box2[0], box2[1], box2[0] + box2[2], box2[1] + box2[3]]

    # Calculate the coordinates of the intersection rectangle
    x_intersection = max(box1[0], box2[0])
    y_intersection = max(box1[1], box2[1])
    w_intersection = min(box1[2], box2[2]) - x_intersection
    h_intersection = min(box1[3], box2[3]) - y_intersection

    # Calculate the area of intersection
    area_intersection = max(0, w_intersection) * max(0, h_intersection)

    # Calculate the area of the union
    area_box1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    area_union = area_box1 + area_box2 - area_intersection

    # Calculate IoU
    iou = area_intersection / area_union

    return iou


In [72]:
box_comparisons = []

for pred_index, pred_row in pred_df.iterrows():
    pred_image_name = pred_row['image_name']
    
    matched=False
    match_box='Background'

    matching_rows = true_df[true_df['image_name'] == pred_image_name]
    
    for true_index, true_row in matching_rows.iterrows():
        print(f"Match found at pred_index {pred_index} and true_index {true_index}")
        print(pred_row)

        #call function to calculate IOU (for 'pred', be sure to use the converted '_pixel' values and not normalized)
        iou_result = calculate_iou(pred_row['x_pixel'], pred_row['y_pixel'], pred_row['w_pixel'], pred_row['h_pixel'],true_row['x'],true_row['y'],true_row['w'],true_row['h'])
        
        print(iou_result)

        if iou_result > 0.5:

            print("now set matched=True and match_box = true_row['class_true']")

    break
        
    #     print("if IOU is >0.5: matched=True, matchbox=true_box.class")
    #     print("make sure this captures all boxes in each image")

    # matched.append() #?


Match found at pred_index 0 and true_index 10
Unnamed: 0                                  0
image_name    7762-4__2019-08-13__09-17-47(1)
class_ID                                   39
x                                    0.485311
y                                    0.377945
w                                    0.192238
h                                    0.104651
confidence                            0.73672
class_pred                             Weasel
width                                    3840
height                                   2880
x_pixel                            1863.59424
y_pixel                             1088.4816
w_pixel                             738.19392
h_pixel                             301.39488
Name: 0, dtype: object
0.1416413071298903


In [ ]:
#loop through
for pred_box in pred_df:
    matched = False
    match_box = "Background"

    for true_box in true_df:
        if pred_box['image_name'].equals(true_box['image_name']):
            print('true')
            break